In [1]:
# IMPORTING LIBRARY
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
import warnings
import datetime
import scipy.stats as stats
warnings.filterwarnings("ignore")

In [4]:
path = r'C:/Users/Mattia/Desktop/DataScience'
df_msft = pd.read_csv(path + '/MSFT.csv', sep=',')
df_msft["Name"] = "MSFT"
df_meta = pd.read_csv(path + '/META.csv', sep=',')
df_meta["Name"] = "META"
df_amzn = pd.read_csv(path + '/AMZN.csv', sep=',')
df_amzn["Name"] = "AMZN"
df_aapl = pd.read_csv(path + '/AAPL.csv', sep=',')
df_aapl["Name"] = "AAPL"

In [5]:
df = pd.concat([df_msft, df_meta, df_amzn, df_aapl], ignore_index=True)
df

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2018-01-02,86.129997,86.309998,85.500000,85.949997,80.940384,22483800,MSFT
1,2018-01-03,86.059998,86.510002,85.970001,86.349998,81.317078,26061400,MSFT
2,2018-01-04,86.589996,87.660004,86.570000,87.110001,82.032784,21912000,MSFT
3,2018-01-05,87.660004,88.410004,87.430000,88.190002,83.049820,23407100,MSFT
4,2018-01-08,88.199997,88.580002,87.599998,88.279999,83.134598,22113000,MSFT
...,...,...,...,...,...,...,...,...
5031,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63771000,AAPL
5032,2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800,AAPL
5033,2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400,AAPL
5034,2022-12-29,127.989998,130.479996,127.730003,129.610001,129.610001,75703700,AAPL


In this way we have created a dataset that allows us to work simply with all the necessary curves

In [6]:
ls = list(df.Name.unique())
for i in df.Name.unique():
    ls.remove(i)
    for j in ls:
        print("Correlation " + i + " - " + j)
        print(str(stats.pearsonr(df[df["Name"]==i]['Adj Close'], df[df["Name"]==j]['Adj Close'])))
        
dc = {
  "AAPL": "MSFT",
  "MSFT": "AAPL",
  "AMZN": "META",
  "META": "AMZN"
}

Correlation MSFT - META
(0.6076976120059763, 5.729596035698225e-128)
Correlation MSFT - AMZN
(0.8397525782734289, 0.0)
Correlation MSFT - AAPL
(0.9738586737751955, 0.0)
Correlation META - AMZN
(0.8391458897797447, 0.0)
Correlation META - AAPL
(0.5100856634147661, 2.4354363376389415e-84)
Correlation AMZN - AAPL
(0.7997949799279774, 6.818765736649285e-281)


We analyse the correlation between the Adjusted Closure of the differents datasets cause it is gonna be useful in the next analysis, when we will create some features

In [7]:
df["lag1_adjclose"] = df.groupby(["Name"])["Adj Close"].diff()
df["diff_opcl"] = df["Open"]-df["Adj Close"]
df["lag1_diff_opcl"] = df.groupby(["Name"])["diff_opcl"].diff()
df = df.groupby(["Name"]).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

df_feat=pd.DataFrame()
for i in df.Name.unique():
    df_name = df[df.Name == i].reset_index(drop = True)
    df_corr = df[df.Name == dc[i]].reset_index(drop = True)
    df_name["corr"] = df_name["Adj Close"] - df_corr["Adj Close"]
    df_feat = df_feat.append(df_name)
df_feat["lag1_corr"] = df_feat.groupby(["Name"])["corr"].diff()
df_feat = df_feat.groupby(["Name"]).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

Here we see the 3 features that we are going to use in our anomaly detection.
The first one is simply the first lag of the adjusted closure
The second one is the difference between the Open price and the Adjusted Closure
The last one is the difference between the couple of Curve with the highest correlation, hence [MSFT,AAPL] and [META,AMZN]

In [8]:
df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,Name,lag1_adjclose,diff_opcl,lag1_diff_opcl,corr,lag1_corr
0,2018-01-04,43.134998,43.367500,43.020000,43.257500,41.133553,89738400,AAPL,0.190194,2.001445,-0.187696,-40.899231,-0.525512
1,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.601864,94640000,AAPL,0.468311,1.758137,-0.243308,-41.447956,-0.548725
2,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.447342,82271200,AAPL,-0.154522,2.140160,0.382023,-41.687256,-0.239300
3,2018-01-09,43.637501,43.764999,43.352501,43.582500,41.442593,86336000,AAPL,-0.004749,2.194908,0.054748,-41.635486,0.051770
4,2018-01-10,43.290001,43.575001,43.250000,43.572498,41.433071,95839600,AAPL,-0.009522,1.856930,-0.337978,-41.268330,0.367156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023,2022-12-23,236.110001,238.869995,233.940002,238.729996,238.729996,21203200,MSFT,0.539994,-2.619995,-5.689988,106.869995,0.909989
5024,2022-12-27,238.699997,238.929993,235.830002,236.960007,236.960007,16688600,MSFT,-1.769989,1.739990,4.359985,106.930008,0.060013
5025,2022-12-28,236.889999,239.720001,234.169998,234.529999,234.529999,17457100,MSFT,-2.430008,2.360000,0.620010,108.489998,1.559990
5026,2022-12-29,235.649994,241.919998,235.649994,241.009995,241.009995,19770700,MSFT,6.479996,-5.360001,-7.720001,111.399994,2.909996


In [11]:
feat = ["lag1_adjclose", "lag1_diff_opcl", "lag1_corr"]
seed = 42
threshold = 0.01
n_estimator = 300
df_tot = pd.DataFrame()

def isolation_forest(feat, df, seed, threshold, n_estimator):
    df_fin = pd.DataFrame()
    model = IsolationForest(contamination=threshold,
                            random_state=seed,
                            n_estimators=n_estimator)                    
    df_array = np.array(df[feat]).reshape(len(df["Adj Close"]), len(feat))
    model.fit(df_array)
    df['anomaly'] = model.predict(df_array)
    df_fin = df_fin.append(df)
    return df_fin

for act in  df_feat.Name.unique():
    df_act = df_feat[df_feat["Name"] == act]
    df_fin = isolation_forest(feat, df_act, seed, threshold, n_estimator)
    df_tot = df_tot.append(df_fin)
df_tot

,Date,Open,High,Low,Close,Adj Close,Volume,Name,lag1_adjclose,diff_opcl,lag1_diff_opcl,corr,lag1_corr,anomaly
0,2018-01-04,43.134998,43.367500,43.020000,43.257500,41.133553,89738400,AAPL,0.190194,2.001445,-0.187696,-40.899231,-0.525512,1
1,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.601864,94640000,AAPL,0.468311,1.758137,-0.243308,-41.447956,-0.548725,1
2,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.447342,82271200,AAPL,-0.154522,2.140160,0.382023,-41.687256,-0.239300,1
3,2018-01-09,43.637501,43.764999,43.352501,43.582500,41.442593,86336000,AAPL,-0.004749,2.194908,0.054748,-41.635486,0.051770,1
4,2018-01-10,43.290001,43.575001,43.250000,43.572498,41.433071,95839600,AAPL,-0.009522,1.856930,-0.337978,-41.268330,0.367156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023,2022-12-23,236.110001,238.869995,233.940002,238.729996,238.729996,21203200,MSFT,0.539994,-2.619995,-5.689988,106.869995,0.909989,1
5024,2022-12-27,238.699997,238.929993,235.830002,236.960007,236.960007,16688600,MSFT,-1.769989,1.739990,4.359985,106.930008,0.060013,1
5025,2022-12-28,236.889999,239.720001,234.169998,234.529999,234.529999,17457100,MSFT,-2.430008,2.360000,0.620010,108.489998,1.559990,1
5026,2022-12-29,235.649994,241.919998,235.649994,241.009995,241.009995,19770700,MSFT,6.479996,-5.360001,-7.720001,111.399994,2.909996,1


Now we will see some graphics representation.
Before we will see the four curves analised, and then we will see the features of one of these (AMZN)

In [15]:
def IF_line_chart(df, name, variable):
    df_plt = df[df.Name == name]
    fig = px.line(df_plt, x="Date", y=variable,
                  title="Plot " + variable + " di " + name,
                  template="simple_white")
    fig.add_trace(go.Scatter(mode="markers",
                             x=df_plt[(df_plt["anomaly"]==-1)].Date,
                             y=df_plt[(df_plt["anomaly"]==-1)][variable],
                             name="anomalies"))
    fig.show()
for i in df.Name.unique():
    IF_line_chart(df_tot.copy(), i ,"Adj Close")

In [18]:
for i in feat:
    IF_line_chart(df_tot.copy(), "AMZN" ,i)


Here we can see some correlation between the data of anomalies.
Firstly we can notice that there is not a date in which all four have an anomalies but there exist one date for each combination of three of them

In [144]:
exc = df_tot[df_tot["ANOMALY"]==-1]
occ = exc.groupby("Date").Name.count().reset_index(drop=False)
for i in range(len(occ)):
    occ["Name"][i] = ",".join(list(exc[exc["Date"]==occ["Date"][i]].Name))
print(occ)
occ.groupby("Name").Date.count()

         Date            Name
0  2018-07-26            META
1  2020-03-13       AAPL,MSFT
2  2020-03-16  AAPL,META,MSFT
3  2020-06-26            META
4  2020-07-20            AMZN
5  2020-07-31  AAPL,AMZN,META
6  2020-08-06            META
7  2020-08-26            META
8  2020-09-03            MSFT
9  2020-11-04            AMZN
10 2021-04-29            META
11 2021-07-23            META
12 2021-07-30            AMZN
13 2021-10-26            META
14 2021-10-27            MSFT
15 2021-11-18            AMZN
16 2021-12-16            AAPL
17 2022-01-13            MSFT
18 2022-01-28            AAPL
19 2022-02-03  AMZN,META,MSFT
20 2022-02-04            AMZN
21 2022-02-24  AAPL,AMZN,MSFT
22 2022-04-20            META
23 2022-04-27       AAPL,MSFT
24 2022-04-28       AMZN,META
25 2022-04-29       AMZN,MSFT
26 2022-05-05       AAPL,AMZN
27 2022-07-12            MSFT
28 2022-07-27       AAPL,MSFT
29 2022-07-29            AMZN
30 2022-09-13            AAPL
31 2022-10-26            MSFT
32 2022-10

Name
AAPL              4
AAPL,AMZN         2
AAPL,AMZN,META    1
AAPL,AMZN,MSFT    1
AAPL,META,MSFT    1
AAPL,MSFT         4
AMZN              6
AMZN,META         1
AMZN,META,MSFT    1
AMZN,MSFT         1
META              9
MSFT              5
Name: Date, dtype: int64